Referências:  
http://cython.readthedocs.io/en/latest/src/tutorial/cython_tutorial.html  
https://numba.pydata.org/  
https://www.ibm.com/developerworks/community/blogs/jfp/entry/Python_Meets_Julia_Micro_Performance?lang=en  
https://gist.github.com/jfpuget/b53f1e15a37aba5944ad  

In [4]:
import random
import time

Criando uma função Fibonacci Recursiva:

In [5]:
def fib(n):
    if n<2:
        return n
    return fib(n-1)+fib(n-2)

In [9]:
%timeit fib(20)
%timeit fib(30)
%timeit fib(35)

3.61 ms ± 24.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
464 ms ± 40 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
5.71 s ± 696 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Compilando a função com o compilador Cython:  
(pip3 install Cython)

In [13]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [14]:
%%cython

def fib_cython(n):
    if n<2:
        return n
    return fib_cython(n-1)+fib_cython(n-2)

In [15]:
%timeit fib_cython(20)
%timeit fib_cython(30)
%timeit fib_cython(35)

1.77 ms ± 314 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
191 ms ± 3.62 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
2.26 s ± 209 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Podemos melhorar ainda mais compilando as variáveis estaticamente. A função deve ser declarada com "cpdef" ao invés de def". Dessa forma, podemos usar os tipos do C como parâmetros da função:

In [16]:
%%cython

cpdef long fib_cython_type(long n):
    if n<2:
        return n
    return fib_cython_type(n-1)+fib_cython_type(n-2)

In [18]:
%timeit fib_cython_type(20)
%timeit fib_cython_type(30)
%timeit fib_cython_type(35)

61.9 µs ± 6.04 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
6.64 ms ± 664 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
77.2 ms ± 11.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Uma outra abordagem é permitindo que os resultados sejam guardados em _cache_:
Dessa forma podemos manter a precisão arbitrária das variáveis do Python, sem recorrer aos tipos estáticos do C

In [28]:
from functools import lru_cache as cache #Python3

@cache(maxsize=None) #Decorador, são modificadores(intermediários) de funções.
def fib_cache(n):
    if n<2:
        return n
    return fib_cache(n-1)+fib_cache(n-2)

In [33]:
%timeit fib_cache(20)
%timeit fib_cache(30)
%timeit fib_cache(35)

201 ns ± 23.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
157 ns ± 20.2 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
146 ns ± 18.6 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


Uma outra maneira de emular o cache seria realizando uma modificação no algoritmo:

In [38]:
def fib_seq(n):
    if n < 2:
        return n
    a,b = 1,0
    for i in range(n-1):
        a,b = a+b,a
    return a

In [39]:
%timeit fib_seq(20)
%timeit fib_seq(30)
%timeit fib_seq(35)

2.4 µs ± 339 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
2.84 µs ± 616 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
4.01 µs ± 809 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Vamos compilar esta nova função com e sem a utilização de tipagem estática: 

In [40]:
%%cython

def fib_seq_cython(n):
    if n < 2:
        return n
    a,b = 1,0
    for i in range(n-1):
        a,b = a+b,a
    return a

In [41]:
%timeit fib_seq_cython(20)
%timeit fib_seq_cython(30)
%timeit fib_seq_cython(35)

1.12 µs ± 195 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
1.26 µs ± 38.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
1.79 µs ± 359 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [44]:
%%cython

cpdef long fib_seq_cython_type(long n):
    if n < 2:
        return n
    cdef long a,b
    a,b = 1,0
    for i in range(n-1):
        a,b = a+b,a
    return a

In [45]:
%timeit fib_seq_cython_type(20)
%timeit fib_seq_cython_type(30)
%timeit fib_seq_cython_type(35)

115 ns ± 14 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
138 ns ± 13.3 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
123 ns ± 15.7 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


Podemos usar uma ferramenta chamada Numba, que é um just-in-time (jit) compiler:

In [46]:
from numba import jit

In [49]:
@jit
def fib_seq_numba(n):
    if n < 2:
        return n
    a,b = 1,0
    for i in range(n-1):
        a,b = a+b,a
    return a

In [50]:
%timeit fib_seq_numba(20)
%timeit fib_seq_numba(30)
%timeit fib_seq_numba(35)

328 ns ± 8.24 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
353 ns ± 38.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
409 ns ± 78.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Segundo exemplo:  
Algoritmo de ordenação Quick Sort  

In [51]:
def qsort_kernel(a, lo, hi):
    i = lo
    j = hi
    while i < hi:
        pivot = a[(lo+hi) // 2]
        while i <= j:
            while a[i] < pivot:
                i += 1
            while a[j] > pivot:
                j -= 1
            if i <= j:
                a[i], a[j] = a[j], a[i]
                i += 1
                j -= 1
        if lo < j:
            qsort_kernel(a, lo, j)
        lo = i
        j = hi
    return a

def benchmark_qsort():
    lst = [ random.random() for i in range(1,5000) ]
    qsort_kernel(lst, 0, len(lst)-1)

Usando Numba:

In [52]:
@jit
def qsort_kernel_numba(a, lo, hi):
    i = lo
    j = hi
    while i < hi:
        pivot = a[(lo+hi) // 2]
        while i <= j:
            while a[i] < pivot:
                i += 1
            while a[j] > pivot:
                j -= 1
            if i <= j:
                a[i], a[j] = a[j], a[i]
                i += 1
                j -= 1
        if lo < j:
            qsort_kernel_numba(a, lo, j)
        lo = i
        j = hi
    return a

def benchmark_qsort_numba():
        lst = [ random.random() for i in range(1,5000) ]
        qsort_kernel_numba(lst, 0, len(lst)-1)

Numba e Numpy:

In [53]:
@jit
def qsort_kernel_numba_numpy(a, lo, hi):
    i = lo
    j = hi
    while i < hi:
        pivot = a[(lo+hi) // 2]
        while i <= j:
            while a[i] < pivot:
                i += 1
            while a[j] > pivot:
                j -= 1
            if i <= j:
                a[i], a[j] = a[j], a[i]
                i += 1
                j -= 1
        if lo < j:
            qsort_kernel_numba_numpy(a, lo, j)
        lo = i
        j = hi
    return a

def benchmark_qsort_numba_numpy():
        lst = np.random.rand(5000)
        qsort_kernel_numba(lst, 0, len(lst)-1)

Usando Numpy e Cython:  

In [54]:
%%cython
import numpy as np
import cython

@cython.boundscheck(False)
@cython.wraparound(False)
cdef double[:] \
qsort_kernel_cython_numpy_type(double[:] a, \
                               long lo, \
                               long hi):
    cdef: 
        long i, j
        double pivot
    i = lo
    j = hi
    while i < hi:
        pivot = a[(lo+hi) // 2]
        while i <= j:
            while a[i] < pivot:
                i += 1
            while a[j] > pivot:
                j -= 1
            if i <= j:
                a[i], a[j] = a[j], a[i]
                i += 1
                j -= 1
        if lo < j:
            qsort_kernel_cython_numpy_type(a, lo, j)
        lo = i
        j = hi
    return a

def benchmark_qsort_numpy_cython():
    lst = np.random.rand(5000)
    qsort_kernel_cython_numpy_type(lst, 0, len(lst)-1)

def benchmark_sort_numpy():
    lst = np.random.rand(5000)
    np.sort(lst)

In [56]:
%timeit benchmark_qsort()
%timeit benchmark_qsort_numba()
%timeit benchmark_qsort_numba_numpy()
%timeit benchmark_qsort_numpy_cython()
%timeit benchmark_sort_numpy()

21.4 ms ± 3.87 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
1.67 ms ± 111 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
646 µs ± 42 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
758 µs ± 30 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
440 µs ± 5.48 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
